# Multiagent colaboration for Financial Analysis

In [1]:
from crewai import Agent, Task, Crew, LLM
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
import os
from dotenv import load_dotenv

/home/avinash/Inexture/AI/Gen AI/Practice/myenv/lib/python3.12/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/home/avinash/Inexture/AI/Gen AI/Practice/myenv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
/home/avinash/Inexture/AI/Gen AI/Practice/myenv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of th

In [2]:
load_dotenv()
serper_api_key = os.getenv("SERPER_API_KEY")
os.environ["SERPER_API_KEY"] = serper_api_key
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [3]:
myllm = LLM(
    model = "gemini/gemini-2.0-flash",
    api_key = os.getenv("GOOGLE_API_KEY")
)

# Creating Agent

In [4]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm = myllm
)

In [5]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm = myllm
)

In [7]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm = myllm
)

# Creating Task

In [8]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [9]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [10]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [11]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

In [12]:
from crewai import Crew, Process


# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm = myllm,
    process=Process.hierarchical,
    verbose=True
)

In [13]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': ' Indian Railway Finance Corporation. ',
    'initial_capital': '10000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [14]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5c682e02-ae58-4196-8436-015e05b79bae                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: de36d48b-4954-4e67-b004-7b26bb87aa31
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: de36d48b-4954-4e67-b004-7b26bb87aa31
       Status: Executing Task...
    └── 🤖 Agent: Crew Manager
            Status: In Progress

# Agent: Crew Manager
## Task: Continuously monitor and analyze market data for the selected stock ( Indian Railway Finance Corporation. ). Use statistical modeling and machine learning to identify trends and predict market movements.


🚀 Crew: crew
└── 📋 Task: de36d48b-4954-4e67-b004-7b26bb87aa31
       Status: Executing Task...
    └── 🤖 Agent: Crew Manager
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Crew Manager
## Final Answer: 
```
**Risk Analysis Report: Indian Railway Finance Corporation (IRFC) Day Trading Strategies**

This report evaluates the risks associated with the proposed day trading strategies and execution plans for Indian Railway Finance Corporation (IRFC) as of April 1, 2025. It provides a detailed analysis of potential risks and suggests mitigation strategies, considering conflicting market signals and available financial data.

**Executive Summary:**

Day trading IRFC presents a mixed bag of opportunities and risks. Conflicting technical signals, a "STRONG SELL" recommendation from some analysts, and fluctuating FII/DII activity necessitate a cautious and well-managed approach. While the company's 'Navratna' status and potential for mean reversion offer potential advantages, traders must be vigilant and adaptable. This report outlines strategy-specific risks and mitigation techniques to aid informed decision-making.

**General Risk Factors:**

*   **Co

🚀 Crew: crew
├── 📋 Task: de36d48b-4954-4e67-b004-7b26bb87aa31
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (1)
│   ├── 🤖 Agent: Data Analyst
│   │       Status: ✅ Completed
│   │   ├── 🔧 Used Search the internet with Serper (1)
│   │   ├── 🔧 Used Read website content (2)
│   │   └── 🔧 Using Ask question to coworker (1)
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (4)
│       └── 🔧 Used Read website content (6)
├── 📋 Task: 90d628d4-4a9a-49c1-b865-c8c108c622f6
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (2)
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       ├── 🔧 Used Delegate work to coworker (2)
│       └── 🔧 Used Search the internet with Serper (5)
├── 📋 Task: cdf83d8a-c6e0-44f9-99aa-b81253a2bfca
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (3)
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (6)
└── 📋 Task: 0f17a43d-ce36-4a05-8c2b-110d8cabd402
       Status: Executing Task...
    ├── 🤖 Agent: Crew Manager
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (4)
    └── 🤖 Agent: Crew Manager
            Status: ✅ Completed
        └── 🔧 Used Search the internet with Serper (7)

🚀 Crew: crew
├── 📋 Task: de36d48b-4954-4e67-b004-7b26bb87aa31
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (1)
│   ├── 🤖 Agent: Data Analyst
│   │       Status: ✅ Completed
│   │   ├── 🔧 Used Search the internet with Serper (1)
│   │   ├── 🔧 Used Read website content (2)
│   │   └── 🔧 Using Ask question to coworker (1)
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (4)
│       └── 🔧 Used Read website content (6)
├── 📋 Task: 90d628d4-4a9a-49c1-b865-c8c108c622f6
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (2)
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       ├── 🔧 Used Delegate work to coworker (2)
│       └── 🔧 Used Search the internet with Serper (5)
├── 📋 Task: cdf83d8a-c6e0-44f9-99aa-b81253a2bfca
│      Assigned to: Crew Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Crew Manager
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (3)
│   └── 🤖 Agent: Crew Manager
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (6)
└── 📋 Task: 0f17a43d-ce36-4a05-8c2b-110d8cabd402
       Assigned to: Crew Manager
       Status: ✅ Completed
    ├── 🤖 Agent: Crew Manager
    │       Status: In Progress
    │   └── 🔧 Using Delegate work to coworker (4)
    └── 🤖 Agent: Crew Manager
            Status: ✅ Completed
        └── 🔧 Used Search the internet with Serper (7)

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0f17a43d-ce36-4a05-8c2b-110d8cabd402                                                                     │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5c682e02-ae58-4196-8436-015e05b79bae                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [17]:
from IPython.display import Markdown
Markdown(result.raw)

```
**Risk Analysis Report: Indian Railway Finance Corporation (IRFC) Day Trading Strategies**

This report evaluates the risks associated with the proposed day trading strategies and execution plans for Indian Railway Finance Corporation (IRFC) as of April 1, 2025. It provides a detailed analysis of potential risks and suggests mitigation strategies, considering conflicting market signals and available financial data.

**Executive Summary:**

Day trading IRFC presents a mixed bag of opportunities and risks. Conflicting technical signals, a "STRONG SELL" recommendation from some analysts, and fluctuating FII/DII activity necessitate a cautious and well-managed approach. While the company's 'Navratna' status and potential for mean reversion offer potential advantages, traders must be vigilant and adaptable. This report outlines strategy-specific risks and mitigation techniques to aid informed decision-making.

**General Risk Factors:**

*   **Conflicting Signals (TradingView vs. Moneycontrol, STRONG SELL recommendation):**
    *   **Risk:** Discrepancies in technical analysis and analyst recommendations can lead to confusion and poor trading decisions.
    *   **Mitigation:** Consult multiple sources, prioritize independent analysis, and understand the biases of different platforms. Acknowledge the bearish bias suggested by the STRONG SELL recommendation and adjust position sizes accordingly.
*   **Market Volatility:**
    *   **Risk:** Unexpected market fluctuations can trigger stop-loss orders and result in losses.
    *   **Mitigation:** Use wider stop-loss orders based on Average True Range (ATR), reduce position sizes, and avoid trading during periods of high market uncertainty.
*   **Information Overload:**
    *   **Risk:** Overwhelming data from various sources can lead to analysis paralysis and delayed decision-making.
    *   **Mitigation:** Focus on a limited set of key indicators and news sources, and develop a clear trading plan.
*   **Black Swan Events:**
    *   **Risk:** Unforeseen events (e.g., policy changes, economic shocks) can drastically impact the stock price.
    *   **Mitigation:** Stay informed about current events and be prepared to adjust positions quickly. Diversify your portfolio to mitigate the impact of any single event.
*   **Decreasing ROE and Improving ROCE:**
    *   **Risk:** While improving ROCE shows better capital allocation, decreasing ROE indicates declining profitability. This contradiction can mislead investors.
    *   **Mitigation:** Investigate the causes of declining ROE and assess if the ROCE improvement sufficiently offsets the risk.

**Strategy-Specific Risk Analysis and Mitigation:**

**Strategy:** Multiple Time Frame Analysis

*   **Risk 1:** Conflicting Signals Across Timeframes
    *   **Mitigation:** Prioritize the longer timeframe trend. Use shorter timeframes for entry/exit timing but align with the dominant trend on the daily or weekly charts. Establish clear rules for resolving conflicting signals (e.g., if the daily trend is up but the hourly shows a pullback, wait for confirmation of upward momentum on the hourly before entering).
*   **Risk 2:** Over-Reliance on Technical Indicators
    *   **Mitigation:** Combine technical analysis with fundamental awareness. Be aware of upcoming news events (e.g., government policy announcements related to railways) that could impact the stock. Set alerts for news releases and adjust positions accordingly.
*   **Risk 3:** Whipsaws and False Signals
    *   **Mitigation:** Implement wider stop-loss orders to account for increased volatility, especially around news events or market open. Use Average True Range (ATR) to determine appropriate stop-loss levels.
*   **Risk 4:** Subjectivity in Trend Identification
    *   **Mitigation:** Use clearly defined criteria for identifying trends (e.g., moving average crossovers, price above/below key support/resistance levels). Backtest these criteria to ensure they are reliable for IRFC.

**Strategy:** Breakout and Breakdown Strategy

*   **Risk 1:** False Breakouts
    *   **Mitigation:** Wait for confirmation of the breakout before entering a trade. This could involve waiting for a retest of the breakout level or looking for increased volume on the breakout. Use price action confirmation.
*   **Risk 2:** High Volatility
    *   **Mitigation:** Reduce position size to account for increased volatility. Use tighter stop-loss orders, but be mindful of false breakouts. Consider using a volatility-based stop-loss (e.g., based on ATR).
*   **Risk 3:** Support and Resistance Levels Not Holding
    *   **Mitigation:** Identify multiple layers of support and resistance. Be aware that key levels may fail, especially during periods of high market volatility or significant news events. Don't trade when major news is expected.
*   **Risk 4:** Gaps After Breakouts/Breakdowns
    *   **Mitigation:** Be aware of the potential for gaps, especially overnight. Adjust entry and stop-loss orders accordingly. Consider using limit orders to enter positions after a gap.

**Strategy:** Moving Average Crossover Strategy

*   **Risk 1:** Lagging Indicators
    *   **Mitigation:** Use faster moving averages to reduce lag, but be aware that this can increase the number of false signals. Experiment with different moving average periods to find the optimal balance for IRFC.
*   **Risk 2:** Whipsaws During Sideways Market
    *   **Mitigation:** Avoid trading moving average crossovers during sideways market conditions. Use other indicators (e.g., ADX) to identify trending markets.
*   **Risk 3:** False Crossovers
    *   **Mitigation:** Require confirmation of the crossover before entering a trade. This could involve waiting for a certain number of periods to pass after the crossover or looking for increased volume.
*   **Risk 4:** Choosing the Wrong Moving Average Periods
    *   **Mitigation:** Backtest different moving average periods to determine which ones have been most effective for IRFC in the past. Optimize the periods based on the current market conditions.

**Strategy:** Mean Reversion Strategy

*   **Risk 1:** Trend Changes
    *   **Mitigation:** Be aware of the overall trend. Mean reversion strategies are best suited for range-bound markets. Avoid using them during strong uptrends or downtrends.
*   **Risk 2:** Identifying the "Mean"
    *   **Mitigation:** Use multiple indicators (e.g., Bollinger Bands, RSI) to identify potential mean reversion opportunities. Backtest these indicators to ensure they are reliable for IRFC.
*   **Risk 3:** Sticking with a Losing Trade for Too Long
    *   **Mitigation:** Set a clear stop-loss order and stick to it. Don't let losing trades run in the hope that the price will eventually revert to the mean.
*   **Risk 4:** Premature Entry
    *   **Mitigation:** Wait for confirmation of a potential mean reversion opportunity before entering a trade. This could involve waiting for a candlestick pattern or looking for divergence between price and an indicator.

**Impact of Key Factors:**

*   **'Navratna' Status:**
    *   **Potential Impact:** Increased investor confidence and improved access to funding could lead to short-term price spikes.
    *   **Risk:** Overvaluing the stock based solely on the 'Navratna' status without considering underlying financials.
    *   **Mitigation:** Monitor news related to the 'Navratna' status but base trading decisions on fundamental and technical analysis.
*   **DII/FII Activity:**
    *   **Potential Impact:** Increased DII buying could provide support, while decreased FII holdings could put downward pressure on the stock.
    *   **Risk:** Misinterpreting institutional activity as a guaranteed trend.
    *   **Mitigation:** Monitor DII and FII activity but consider other factors before making trading decisions.
*   **High Promoter Holding (86.36%):**
    *   **Potential Impact:** Reduced free float can lead to increased volatility.
    *   **Risk:** Price manipulation and limited liquidity.
    *   **Mitigation:** Be aware of the potential for volatility and adjust position sizes accordingly.

**Risk Management Recommendations:**

*   **1% Rule:** Risk no more than 1% of your total trading capital on any single trade.
*   **Stop-Loss Orders:** Mandatory for every trade, placed at predetermined levels based on market volatility and risk tolerance.
*   **Position Sizing:** Calculate position size based on entry price, stop-loss level, and 1% risk tolerance.
*   **Diversification:** Diversify your portfolio across different stocks and asset classes to mitigate the impact of losses on any single trade.
*   **Continuous Monitoring:** Continuously monitor market conditions, news events, and financial data, and be prepared to adjust your trading plan accordingly.
*   **Backtesting:** Backtest trading strategies using historical data to assess their effectiveness and identify potential weaknesses.
*   **Paper Trading:** Practice trading strategies using a demo account before risking real capital.
*   **Stay Informed:** Keep abreast of the latest news and developments related to IRFC and the Indian railway sector.

**Conclusion:**

Day trading IRFC involves inherent risks that must be carefully managed. By understanding the strategy-specific risks, implementing appropriate mitigation techniques, and adhering to sound risk management principles, traders can improve their chances of success. It is crucial to remain adaptable, stay informed, and continuously refine your trading plan based on market conditions and new information.

**Disclaimer:**

This risk analysis report is for informational purposes only and does not constitute financial advice. Trading in the stock market involves risk, and you could lose money. Consult with a qualified financial advisor before making any trading decisions.
```